In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import pandas as pd

In [2]:
# open a new window with the later link
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\wongl\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


# Nasa Mars News

Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest) and collect teh latest News Title and Paragraph Text. Assing the text to variables that you can reference later.

In [3]:
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)

In [4]:
html = browser.html
soup = bs(html,'html.parser')

In [5]:
# had to use a different method to get info because the .find 
# either wouldn't return the correct info or wouldn't return any info for:
# soup.find('li',class_='slide')
# soup.find('div',class_='content-title')
# soup.find('a')

In [6]:
# There’s also a method called select_one(), which finds only the first tag that matches a selector
slide = soup.select_one("ul.item_list li.slide")
title = slide.find('div',class_='content_title').text
para = slide.find('div',class_='article_teaser_body').text
print(f"{title}\n{para}")

NASA's Perseverance Rover Is Midway to Mars 
Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination.


# JPL Mars Space Images - Featured Image

* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

* Make sure to find the image url to the full size .jpg image.

* Make sure to save a complete url string for this image.

In [7]:
# open the new testing page for the image
marsImg_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(marsImg_url)

In [8]:
#splinter: image located in the full image > 'full_image' button

# #giving error: 'WebDriver' object has no attribute 'find' or returns nothing
# fullImg_button = soup.find('a',id_='full_image')

fullImg_button = browser.find_by_id('full_image')
# clicks the button and brings us to the full image
fullImg_button.click()

In [9]:
# splinter: image located in more info > 'more info' button
moreInfo_button = browser.links.find_by_partial_text('more info')
moreInfo_button.click()

In [10]:
# had to move this below the above items
# wouldn't work when not on the same page as the extraction
html = browser.html
img_soup = bs(html,'html.parser')
# image located in img element with class 'main_image'
img_url = img_soup.find('figure',class_='lede').find('a').get('href')
img_url = 'https://www.jpl.nasa.gov'+img_url
print(f"{img_url}")

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17171_hires.jpg


# Mars Facts

* Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [11]:
marsFacts_url = 'https://space-facts.com/mars/'
browser.visit(marsFacts_url)

In [12]:
facts_table = pd.read_html(marsFacts_url)
facts_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [13]:
# use index 0 since it returns a string of values
mars_df = facts_table[0]
# rename the columns
mars_df.columns = ['Fact','Value']
mars_df

,Fact,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


# Mars Hemispheres

* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [14]:
marsHem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(marsHem_url)

In [15]:
html = browser.html
hem_soup = bs(html,'html.parser')
# collect the item containers
hem_url = hem_soup.find_all('div',class_='item')
# hem_url

In [16]:
hem_dict = []

for hem in range(len(hem_url)):
    hem_item = {}    
    
    # click on each of the links
    browser.find_by_css('a.product-item h3')[hem].click()
    
    # get the enhanced image title
    hem_item["img_title"] = browser.find_by_css('h2.title').text
    # get the enhanced image link
    hem_item["img_url"] = browser.links.find_by_text('Sample')['href']
        
    # add it to the hemisphere dictionary
    hem_dict.append(hem_item)
    
    # have to go back to the main browser page
    browser.back()

print(hem_dict)

[{'img_title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'img_title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'img_title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'img_title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


# Quit and close the browser

In [17]:
browser.quit()